In [3]:

import datetime

import netCDF4
import xarray as xr
import cdsapi
import warnings
import matplotlib.pyplot as plt
plt.style.use('default')
import cftime

from shapely.geometry import box, mapping
import geopandas as gpd

def convert_to_180(ds):
    return ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')

# Here, we use the ERA5 data and the Python API interface to cdsapi.
# The script downloads ERA5 used to compare with the calculated values of the RTM model.
# The difference between variables is used to bias-correct the RTM model. To use this script you need to install the cdsapi library
# and to have a valid account and credentials. The data is downloaded in netCDF format and saved in the specified path.
#
# https://www.ecmwf.int/sites/default/files/elibrary/2015/18490-radiation-quantities-ecmwf-model-and-mars.pdf
# ERA5, Monthly Means of Daily Forecast Accumulations
# https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means?tab=form
#
# Trond Kristiansen, May 11 2024

def get_global_era5():
    c = cdsapi.Client()

    years = ['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010',
             '2011','2012','2013','2014','2015','2016','2017','2018','2019']

    c.retrieve(
    'reanalysis-era5-single-levels-monthly-means',
    {
        'product_type': 'monthly_averaged_reanalysis',
        'variable': [
            'mean_surface_downward_uv_radiation_flux', 'mean_surface_net_short_wave_radiation_flux',
        ],
        'year': [
            '1979', '1980', '1981',
            '1982', '1983', '1984',
            '1985', '1986', '1987',
            '1988', '1989', '1990',
            '1991', '1992', '1993',
            '1994', '1995', '1996',
            '1997', '1998', '1999',
            '2000', '2001', '2002',
            '2003', '2004', '2005',
            '2006', '2007', '2008',
            '2009', '2010', '2011',
            '2012', '2013', '2014',
            '2015', '2016', '2017',
            '2018', '2019', '2020',
            '2021', '2022', '2023',
            '2024',
        ],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'time': '00:00',
        'area': [
            90, -180, 50,
            180,
        ],
        'format': 'netcdf',
    },
    "/mnt/disks/actea-disk-1/light/era5/ERA5_global_mean_shortwave_uvb.nc")

In [4]:
%%time
get_global_era5()

2024-09-13 18:02:54,315 INFO Welcome to the CDS.
As per our announcements on the Forum, this instance of CDS will be decommissioned on 26 September 2024 and will no longer be accessible from this date onwards.
Please update your cdsapi package to a version >=0.7.2, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
2024-09-13 18:02:54,316 WARNING MOVE TO CDS-Beta
2024-09-13 18:02:54,317 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2024-09-13 18:02:54,541 INFO Request is completed
2024-09-13 18:02:54,542 INFO Downloading https://download-0002-clone.copernicus-climate.eu/cache-compute-0002/cache/data8/adaptor.mars.internal-1726012338.4677591-11064-12-cb8335ef-948c-4f90-85e2-659c53a314f9.nc to /mnt/disks/actea-disk-1/light/era5/ERA5_global_mean_shortwave_uvb.nc (969.3M)
2024-09-13 18:08:33,733 INFO Download rate 2

CPU times: user 13.2 s, sys: 3.29 s, total: 16.5 s
Wall time: 5min 40s
